<a href="https://colab.research.google.com/github/dilshankumarage/Customer_Churn_Prediction/blob/main/Customer_Churn_Prediction_withui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("/Telco-Customer-Churn.csv")
df.drop("customerID", axis=1, inplace=True)
df["Churn"] = df["Churn"].map({"Yes": 1, "No": 0})
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce")
df["TotalCharges"].fillna(df["TotalCharges"].mean(), inplace=True)
df = pd.get_dummies(df, drop_first=True)

from sklearn.model_selection import train_test_split

X = df.drop("Churn", axis=1)   # features
y = df["Churn"]               # target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)


joblib.dump(model, "churn_model.pkl")
joblib.dump(X.columns, "model_columns.pkl")

/tmp/ipython-input-949884049.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["TotalCharges"].fillna(df["TotalCharges"].mean(), inplace=True)
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver 

['model_columns.pkl']

In [ ]:
import ipywidgets as widgets
from IPython.display import display
import pandas as pd
import joblib

model = joblib.load("churn_model.pkl")
columns = joblib.load("model_columns.pkl")

tenure = widgets.IntSlider(value=12, min=0, max=72, description="Tenure")

monthly_charges = widgets.FloatText(value=70.0, description="Monthly Charges")
total_charges = widgets.FloatText(value=1000.0, description="Total Charges")

contract = widgets.Dropdown(
    options=["Month-to-month", "One year", "Two year"],
    description="Contract"
)

internet = widgets.Dropdown(
    options=["DSL", "Fiber optic", "No"],
    description="Internet"
)

payment = widgets.Dropdown(
    options=[
        "Electronic check",
        "Mailed check",
        "Bank transfer (automatic)",
        "Credit card (automatic)"
    ],
    description="Payment"
)

button = widgets.Button(description="Predict Churn")
output = widgets.Output()

def predict_churn(b):
    with output:
        output.clear_output()

        input_data = pd.DataFrame(0, index=[0], columns=columns)

        input_data["tenure"] = tenure.value
        input_data["MonthlyCharges"] = monthly_charges.value
        input_data["TotalCharges"] = total_charges.value

        if contract.value == "One year":
            input_data["Contract_One year"] = 1
        elif contract.value == "Two year":
            input_data["Contract_Two year"] = 1

        if internet.value == "Fiber optic":
            input_data["InternetService_Fiber optic"] = 1
        elif internet.value == "No":
            input_data["InternetService_No"] = 1

        payment_col = f"PaymentMethod_{payment.value}"
        if payment_col in input_data.columns:
            input_data[payment_col] = 1

        prediction = model.predict(input_data)[0]
        probability = model.predict_proba(input_data)[0][1]

        if prediction == 1:
            print(f"⚠️ Customer likely to CHURN (Probability: {probability:.2f})")
        else:
            print(f"✅ Customer likely to STAY (Probability: {probability:.2f})")

button.on_click(predict_churn)

display(
    tenure,
    monthly_charges,
    total_charges,
    contract,
    internet,
    payment,
    button,
    output
)


IntSlider(value=12, description='Tenure', max=72)

FloatText(value=70.0, description='Monthly Charges')

FloatText(value=1000.0, description='Total Charges')

Dropdown(description='Contract', options=('Month-to-month', 'One year', 'Two year'), value='Month-to-month')

Dropdown(description='Internet', options=('DSL', 'Fiber optic', 'No'), value='DSL')

Dropdown(description='Payment', options=('Electronic check', 'Mailed check', 'Bank transfer (automatic)', 'Cre…

Button(description='Predict Churn', style=ButtonStyle())

Output()